In [15]:
import random
import csv
from collections import Counter

def simulate_gacha(next_is_guarantee):
    """In Honkai: Star Rail:
    When you hit a 5-star, you have a 50% chance to get the featured (rate-up) character.
    If you lose the 50/50, the next 5-star is guaranteed to be the rate-up."""
    if next_is_guarantee:
        # If the next 5-star is guaranteed, you will always win the rate-up character
        # After winning, the next 5-star will not be guaranteed
        return "Win", False  # Win the rate-up character, next is not guaranteed
    else:
        # If not guaranteed: do a 50/50 chance roll
        if random.random() < 0.5:
            return "Win", False
        else:
            return "Lose", True
        
def simulate_five_star(next_is_guarantee):
    pity = 0
    while pity < 90:
        pity += 1
        if pity >= 75:
            chance = 0.006 * (pity - 74)
        else:
            chance = 0.0

        base_chance = 0.003
        total_chance = base_chance + chance

        if random.random() < total_chance:
            result, next_is_guarantee = simulate_gacha(next_is_guarantee)
            return pity, result, next_is_guarantee
    # If no 5-star is triggered after 90 pulls, force a 5-star at 90th pull
    result, next_is_guarantee = simulate_gacha(next_is_guarantee)
    return pity, result, next_is_guarantee
        
pity_used, result, next_is_guarantee = simulate_five_star(False)
print(f"Pity used: {pity_used}, Result: {result}, Next is guarantee: {next_is_guarantee}")

trial_runs = 10000
wins = 0
losses = 0
pity_count = []

next_is_guarantee = False
for i in range(trial_runs):
    pity_used, result, next_is_guarantee = simulate_five_star(next_is_guarantee)
    pity_count.append(pity_used)
    
    if result == "Win":
        wins += 1
    else:
        losses += 1

average_pity = sum(pity_count) / trial_runs
max_pity = max(pity_count)
min_pity = min(pity_count)

print(f"After {trial_runs} trials:")
print(f"Average pulls used: {average_pity:.2f}")
print(f"Max pulls used (worst case): {max_pity}")
print(f"Min pulls used (best case): {min_pity}")
print(f"50/50 Wins: {wins} ({wins / trial_runs:.2%})")
print(f"50/50 Losses: {losses} ({losses / trial_runs:.2%})")

# ---
# Exporting the character banner results to a CSV file
# ---

# Use Counter to get the frequency of each pity value
c = Counter(pity_count)

# Open the CSV file and write the data
with open("sim_character_summary.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["pity", "count"])
    for pity, count in sorted(c.items()):
        writer.writerow([pity, count])

print("\nCharacter simulation results exported to sim_character_summary.csv")

Pity used: 79, Result: Win, Next is guarantee: False
After 10000 trials:
Average pulls used: 76.50
Max pulls used (worst case): 90
Min pulls used (best case): 1
50/50 Wins: 6673 (66.73%)
50/50 Losses: 3327 (33.27%)

Character simulation results exported to sim_character_summary.csv
